# First example with HDF5

In [23]:
import h5py
import ujson
import gzip
import ast
from pandas import DataFrame
from toolz import dissoc
from toolz import dissoc, partition_all
from castra import Castra
import time
import datetime
import dask.dataframe as dd
import dask.bag as db
from dask.diagnostics import ProgressBar
import numpy as np
import pandas as pd
from pandas import HDFStore

In [53]:
path_to_file = "/home/ec2-user/amazon_dataset/data.hdf5"
df = pd.DataFrame(np.random.randn(10, 5))
current_index = 10 #Starting from 0, pointing to the first "free" position
chunks = 10 #writing 10 lines each time

with h5py.File(path_to_file, "w") as hf:
    hf.create_dataset('amazon', data=df,maxshape=(None,None))

In [54]:
with h5py.File(path_to_file, "r") as hf:
    print 'List of arrays in this file: \n', hf.keys()
    data = hf.get('amazon')
    np_data = np.array(data)
    print'Shape of the array amazon: \n', np_data.shape

List of arrays in this file: 
[u'amazon']
Shape of the array amazon: 
(10, 5)


Trying to reshape and add more content to the HDF5:

In [55]:
with h5py.File(path_to_file, "r+") as hf:
    data = hf.get('amazon')
    df = pd.DataFrame(np.random.randn(10, 5)) #New dataframe to add
    data.resize((data.shape[0]+chunks,data.shape[1]))
    data[current_index:current_index+chunks] = df
    print 'List of arrays in this file: \n', hf.keys()
    print 'Hierarchy:',data.name
    np_data = np.array(data)
    print'Shape of the array amazon: \n', np_data.shape
    print 'Data:\n',data[:]

List of arrays in this file: 
[u'amazon']
Hierarchy: /amazon
Shape of the array amazon: 
(20, 5)
Data:
[[ 0.33615614  0.92148885  0.80791913 -0.07501982  1.50430268]
 [ 0.7479168   0.65582071 -0.71301003  0.9000312   2.12067986]
 [ 0.28266367 -0.93705573 -0.93001321 -0.04424039  0.37498966]
 [ 0.69727914  0.88918464 -2.12194092  1.03283473 -0.23717406]
 [-0.22126272 -0.18595924  0.70276079 -0.25828817  0.03022614]
 [-0.25548966 -0.50407185 -0.68975554  0.67881699  0.375861  ]
 [ 1.68238172 -0.94102896 -0.91619115 -0.22347315 -0.48222473]
 [ 0.44037292  0.1177223   0.73702812 -0.71398744  1.0187634 ]
 [-0.99370112 -0.366224   -0.56279895  0.26389202 -0.59253603]
 [-0.71683573  1.2921071  -1.59845757  0.719606   -0.8612614 ]
 [ 0.72463843  0.07863037  0.00382157 -1.39810272 -1.81331261]
 [-0.45035738  0.7919192  -0.40131715  0.12347294 -0.81692247]
 [ 0.07909288  1.13137146  1.43555496 -1.92644076  1.05954108]
 [ 0.68715044 -0.55307139 -1.49701966 -1.61766041  0.59391732]
 [-2.79255144 -

In [27]:
dd.read_hdf(path_to_file,'/amazon')

ValueError: Input must be a list longer than 0

Now creating the Dask DataFrame:

In [146]:
#First DF
df = pd.DataFrame(np.random.randn(10, 5))
#Create HDF5
df.to_hdf(path_to_file,'/amazon',format='table')

#Second DF
df = pd.DataFrame(np.random.randn(10, 5))
df.to_hdf(path_to_file,'/amazon',format='table',append=True)

#Third DF
df = pd.DataFrame(np.random.randn(10, 5))
df.to_hdf(path_to_file,'/amazon',format='table',append=True)

#Check status
with h5py.File(path_to_file, "r+") as hf:
    data = hf.get('amazon/table')
    print data.shape
    print data[:]

(30,)
[ (0, [0.897718067236105, -1.0946661914237923, 2.0314060744354916, 0.37843178259651294, 1.8832041047888866])
 (1, [-0.28564043136680733, -0.07655075474193072, 0.540224405128584, -0.35796842299738124, -0.9406625980997961])
 (2, [0.3205245813708483, -0.7609832345599711, -0.5724229802992692, 0.7995682782984899, 0.39250741780216425])
 (3, [0.9428039707471949, 0.702020781965266, -0.05146169323140638, 0.8946086778295255, 0.37578254621924284])
 (4, [-0.23418750846014688, 0.7638746474643954, -1.1318622671203682, 1.5617540487978496, -0.6106827553232297])
 (5, [-0.4677923345600671, 1.1000987559518314, 0.38678789045171685, -0.93770514543715, 0.18821073801108623])
 (6, [2.188181894477686, -0.30098460344717487, 0.13535327088522206, 1.2107808149056434, 0.5053363211870221])
 (7, [-0.8534687441165866, 1.3112811679624061, -1.0672488905501138, 0.8436103332715101, -0.23720971404964686])
 (8, [-0.014267216760865736, -2.439805782094342, -0.16067586652974358, 0.2510792452943708, -0.833267244505457])
 

In [45]:
df = pd.DataFrame(np.random.randn(10, 5))
df.to_hdf(path_to_file,'/amazon',format='table')
ddf = dd.read_hdf(path_to_file,key='/amazon')
with h5py.File(path_to_file, "r+") as hf:
    data = hf.get('amazon/table')
    print data[:]
    df = pd.DataFrame(np.random.randn(10, 5))
    print data.shape
    data.resize((data.shape[0]+chunks,))
    print data.shape
    data[current_index:current_index+chunks] = df
    print 'Data:\n',data[:]

[ (0, [0.8431498700118035, -0.26071150407298455, -0.17717400333493152, 0.972291626015662, -1.9821576100992873])
 (1, [0.25442721137162405, -0.9508859044686622, -0.6912219066515594, 1.696716124429185, -0.12900855920293555])
 (2, [0.7609659073693733, -0.1636926592643042, 1.5539749829369545, 1.6315731234975972, -0.7995324229487413])
 (3, [-0.27556138904929234, 0.05216452252019807, -3.3079324342059646, -0.46611052296224664, 1.2176568561745245])
 (4, [-1.6228427558698009, -0.5702520598240428, 0.49270022069764885, 0.7664246954510092, -1.5151507755045541])
 (5, [-0.47711779501545337, -1.768899129071236, -0.21830344536814678, 2.0949026925302703, -0.5553597531906278])
 (6, [-0.9539068208163176, -2.2245187760419403, -0.8068229606373949, 1.1477310920064447, -1.5644619776974054])
 (7, [-1.0099870383961342, 0.8550006144060289, 0.4543298642506465, 0.3700122002012492, 1.2046136946214707])
 (8, [-0.29723422304075203, 0.2995589052986324, -1.2861135643338708, -1.8224759858498658, -0.5396499969987226])
 

TypeError: Can't broadcast (10, 5) -> (10,)

In [110]:
ddf.compute()

,0,1,2,3,4
0,-0.526320,-0.085739,-1.037743,1.025857,0.873915
1,-0.235845,0.784827,-0.932460,-0.672638,-1.338503
2,0.446893,0.084582,0.160157,-0.445192,-1.402381
3,0.184008,-0.193218,-0.272635,-1.414573,0.524036
4,-0.961818,1.926870,0.943832,0.347053,0.923796
5,-0.791976,-0.918043,0.300772,0.310206,-0.238695
6,0.528471,0.684842,0.568258,0.786830,-0.107109
7,-1.252557,0.377502,1.178489,0.920960,-0.804955
8,1.152096,-1.653841,2.309369,-2.517307,-1.423715
9,0.681187,-2.534503,1.879892,1.621802,-1.031088


# DataFrames with data from JSON files

In [137]:
path_to = "/home/ec2-user/amazon_dataset/"
f = 'reviews_Musical_Instruments_5.json'
reviews_columns = ['asin', 'reviewerID','reviewerName', 'overall','summary','reviewText','reviewTime','unixReviewTime']
metadata_columns = ['asin','title','price','imUrl','related','also_bought','also_viewed','bought_together','salesRank','brand','categories']
str_dt = h5py.special_dtype(vlen=str)
#dtypes = np.dtype([('asin',str_dt),('reviewerID',str_dt),('reviewerName',str_dt),('overall',float),('summary',str_dt),
 #                  ('reviewText',str_dt),('reviewTime',str_dt),('unixReviewTime',int)])
chunksize = 5000
current_index = 0

#Convert a line of JSON into a cleaned up dict.
def to_json(line):
    return ujson.loads(line.encode('utf8'))

#Convert a not proper line of JSON (due to single quotes) into a cleaned up dict.
def fix_json(line):
    return ast.literal_eval(line)

#Convert a list of JSON strings into a DataFrame
def to_df(batch,filename):
    if filename == 'metadata':
        blobs = map(fix_json,batch)
        df = DataFrame.from_records(blobs, columns=metadata_columns)
    else:
        blobs = map(to_json, batch)
        df = DataFrame.from_records(blobs, columns=reviews_columns)
    return df

def create_new(path_to_file,df, current_index):
    with h5py.File(path_to_file, "w") as hf:
        hf.create_dataset('amazon', data=df,maxshape=(None,None),dtype=str_dt)
        current_index+=chunksize
    return hf, current_index

def extend_hdf5(path_to_file,df, current_index, chunksize):
    with h5py.File(path_to_file, "r+") as hf:
        data = hf.get('amazon')
        if len(df)<chunksize:
            chunksize=len(df)
        data.resize((data.shape[0]+chunksize,data.shape[1]))
        data[current_index:current_index+chunksize] = df
        current_index+=chunksize
        return current_index

def create_hdf5(fullpath,chunksize,current_index):
    filename = fullpath.split('/')[-1].split('.')[0]
    with open(fullpath,'r') as f:
        batches = partition_all(chunksize, f)
        store = None
        for batch in batches:
            df = to_df(batch,filename)
            if store==None:
                store, current_index = create_new(path_to_file,df,current_index)
            else:
                current_index = extend_hdf5(path_to_file,df, current_index, chunksize)

In [138]:
create_hdf5(path_to+f,chunksize,current_index)

In [144]:
ddf = dd.read_hdf(path_to_file,'/amazon')

ValueError: Input must be a list longer than 0

In [179]:
path_to = "/home/ec2-user/amazon_dataset/"
f = 'reviews_Musical_Instruments_5.json'
reviews_columns = ['asin', 'reviewerID','reviewerName', 'overall','summary','reviewText','reviewTime','unixReviewTime']
metadata_columns = ['asin','title','price','imUrl','related','also_bought','also_viewed','bought_together','salesRank','brand','categories']
str_dt = h5py.special_dtype(vlen=unicode)
#dtypes = np.dtype([('asin',str_dt),('reviewerID',str_dt),('reviewerName',str_dt),('overall',float),('summary',str_dt),
 #                  ('reviewText',str_dt),('reviewTime',str_dt),('unixReviewTime',int)])
chunksize = 5000
current_index = 0

#Convert a line of JSON into a cleaned up dict.
def to_json(line):
    return dissoc(ujson.loads(line.encode('utf8')),'reviewerName')

#Convert a not proper line of JSON (due to single quotes) into a cleaned up dict.
def fix_json(line):
    return ast.literal_eval(line)

#Convert a list of JSON strings into a DataFrame
def to_df(batch,filename):
    if filename == 'metadata':
        blobs = map(fix_json,batch)
        df = DataFrame.from_records(blobs, columns=metadata_columns)
    else:
        blobs = map(to_json, batch)
        df = DataFrame.from_records(blobs, columns=reviews_columns)
    return df

def create_new(path_to_file,df, current_index):
    types = df.apply(lambda x: pd.lib.infer_dtype(x.values))
    for col in types[types=='unicode'].index:
        df[col] = df[col].astype(str)
    df.columns = [str(c) for c in df.columns]
    df.to_hdf(path_to_file,'/amazon',format='table',data_columns=True)
    current_index+=chunksize
    return hf, current_index

def extend_hdf5(path_to_file,df, current_index, chunksize):
    df.to_hdf(path_to_file,'/amazon',format='table',data_columns=True,append=True)
    current_index+=chunksize
    return current_index

def create_hdf5(fullpath,chunksize,current_index):
    filename = fullpath.split('/')[-1].split('.')[0]
    with open(fullpath,'r') as f:
        batches = partition_all(chunksize, f)
        store = None
        for batch in batches:
            df = to_df(batch,filename)
            if store==None:
                store, current_index = create_new(path_to_file,df,current_index)
            else:
                current_index = extend_hdf5(path_to_file,df, current_index, chunksize)

In [180]:
create_hdf5(path_to+f,chunksize,current_index)

#Check status
with h5py.File(path_to_file, "r+") as hf:
    data = hf.get('amazon/table')
    print data.shape
    print data[:10]

/home/ec2-user/anaconda2/lib/python2.7/site-packages/tables/leaf.py:386: PerformanceWarning: The Leaf ``/amazon/_i_table/reviewText/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)


TypeError: [unicode] is not implemented as a table column

In [65]:
df = pd.DataFrame([['a','1'],['b','2']], columns=['x','y'])
df.dtypes

x    object
y    object
dtype: object

In [63]:
df.convert_objects(convert_dates=True,convert_numeric=True,convert_timedeltas=True).dtypes

/home/ec2-user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


x    object
y     int64
dtype: object

In [134]:
df = pd.DataFrame([['a','1'],['b','2']], columns=['x','y'])
len(df)

2